In [5]:
# TODO: 
# - write parameters into a text file to recall them later
# - create auditory chain method:
#      * which python music player?
#      * 
# - create Constants class?
# - create gui:
#      * draw layout/contents/functionality on paper (do it like a pro instead of a blockhead for once)
#      * ???
#      * profit
# - brain vision recorder LSL
# - extend marker class?
#
#

import random
import numpy as np

# not sure if this is useful
class Marker:
    
    ################ standard definitions ######################
    def __init__(self,identifier):
        self.value = identifier
        
    def __str__(self):
        return 'Marker ' + str(self.value)
    
    def __repr__(self):
        return 'Marker ' + str(self.value)
    
    ################# CONTENT            #######################
        
    def identify(self): return self.value

class Sequence:
    
        def __init__(self,probt,probsd,length,prenov):
            self.sequence = []
            self.expandedSeq = []
            self.markerCount = 0
            self.amount_Target = 0
            self.amount_Novelty = {}
            self.amount_Standard = 0
        
            self.trails(probt,probsd,length,prenov)
            if self.sequence == []:
                raise Exception
        
        def trails(self,Prob_T, Probs_D,Length,PreNov):

            amount_T = round(Length * Prob_T) # get amount of Targets from propability of a Target + amount of stimuli 
            amount_Ds = {} # fill dictonary with the amounts of every distractor stimuli same as above
            
            # this will be the sum of all probabilities but the standard one, so I can see if it 
            # is even possible to create a sequence
            probs_non_standard = Prob_T
            
            
            for name, prob in Probs_D.items():   
                amount = round(prob*Length)
                amount_Ds[name] = amount
                probs_non_standard += prob 

            if probs_non_standard * PreNov > 1: 
                print(self.errorMsg(probs_non_standard,PreNov))
                self.sequence = [] # TODO: raise exception here
            '''
            this happens if you for example say you want 2 standard stimuli previous to each novelty stimuli, the whole thing
            has to be 10 stimuli long and the two novelty stimuli have to appear with a probability of 0.4
            => length 10 => 8 novelty stimuli, each has to succeed 2 standard stimuli => 3 * 8 == 24 > 10!
            => that's simply not possible
            '''
            
            # save these values so we can call them outside of this function
            self.amount_Target = amount_T
            self.amount_Novelty = amount_Ds
            self.amount_Standard = Length - amount_T - sum(amount_Ds.values())

            sequence, amount_S = self.makeSeqs(PreNov,amount_T,amount_Ds,Length) 
            ''' 
            takes the amount of standard stimuli previous to novelty stimuli, the amount of target and distractor stimuli
            and the total amount of presented stimuli and returns a list of atomic sequences and the amount of standard stimuli
            which are still free to distribute
            '''



            if amount_S == 0: self.sequence =  self.permutations(sequence)
            '''
            replicate(0,[['S']]) == [['S']], there are no monoid structures (I think) in python so 
            I have to solve it likes this sadly
            '''

            self.sequence = self.permutations(replicate(amount_S,[['S']])+sequence)
            '''
            we 'randomly' choose one permutation of the list of atomic sequences
            and return it
            '''


        def makeSeqs(self,pn,aT,aDs,length):

            seq_S = replicate(pn,['S'])

            seq_T = seq_S + ['T']
            seq_Ds = []

            for name, amount in aDs.items():
                seq_Ds += replicate(amount,[seq_S + [name]])

            result = seq_Ds + replicate(aT,[seq_T])
            return result, (length - (len(result) * (pn+1)))

        # returns one possible 'random' permutation of the given list
        def permutations(self,list):
            permus = random.sample(list,len(list))    
            return flatten(permus)

        # simply creates the error message when the given parameters in trails dont fit
        def errorMsg(self,probs,prenov):
            maxPreNov = round(1 / probs)
            maxProbs = float('%.2f'%(1 / prenov))
            return '\nIt is not possible to create a sequence of stimuli with the given probabilites for novelty stimuli and the given amount of buffer previous to novelty stimuli!\
                    \nEither change the buffer to {0} at max or the sum of novelty probabilites to {1} at max'.format(maxPreNov,maxProbs)
        
        
        # write oddball steps into a list, creating a sequence of actions which then can be 
        # easily executed
        # ISI = inter stimulus interval, PSI = pre stimulus interval, 
        # scaling_jitter = scaling factor for standard distribution, make it bigger!!!
        def expand(self,ISI,PSI,standard_dev,maxValue_jitter,minValue_jitter,mean_jitter,scaling_jitter):
            # I only call the expand_sequence method here, see below
            self.expandedSeq = self.expand_sequence(
                                               ISI,
                                               PSI,
                                               standard_dev,
                                               maxValue_jitter,
                                               minValue_jitter,
                                               mean_jitter,
                                               scaling_jitter)
            
        # maps te expand_single function over the whole sequence
        # since x consists of the pattern [[...],[...],...,[...]]
        # I have to flatten it into [.....]
        def expand_sequence(self,*args):
            x = map(lambda i: self.expand_single(i,*args),self.sequence)
            return flatten(list(x))

        # just concatinate the values 
        def expand_single(self,piece,ISI,PSI,sd,mav,miv,mean,scaling):
            realISI = ISI - PSI
            return ([self.marker()] +
                    [PSI]           + 
                    [self.marker()] + 
                    [piece]         +  
                    [self.marker()] + 
                    [realISI+self.calc_jitter(sd,mav,miv,mean,scaling)]
                   )

        # pulls a value from the gausian standard distribution, magnifies it with the 
        # scaling factor and then trunctuates it with the max and min values
        def calc_jitter(self,standard_dev, max_val, min_val, mean, scaling):
            return minmax(min_val,
                          max_val,
                          round(scaling*np.random.normal(mean,standard_dev)))
        
        # creates a Marker (see above, far above)
        def marker(self):
            curm = self.markerCount
            self.markerCount = curm+1
            return Marker(curm)
        
        ####### GETTERS ########
        
        def getTargetAmount(self):   return self.amount_Target
        def getStandardAmount(self): return self.amount_Standard
        def getNoveltyAmount(self):  return self.amount_Novelty        
        def getSeq(self):            return self.sequence
        def getExpandedSeq(self):    return self.expandedSeq
        def getMarkerCount(self):    return self.markerCount
        
# concatinates a list of lists
def flatten(list):
    return [e for sublist in list for e in sublist]
    # how to read it:
    # for each sublist in the given list, for each element in the sublist, put that element into a list
    
# replicate(Int n,Listtype e): returns the e concatenated n times with itself
# replicate(5,'S') == 'SSSSS'
def replicate(n,e): 
    if n <= 1: return e
    k = replicate(n-1,e)
    return e + k

# rounds a value up or down if it is not in between the given min value and max value
# minmax(5,10,6) = 6; minmax(5,10,11) =  10
def minmax(minv,maxv,x):
    t1 = min(x,maxv)
    t2 = max(t1,minv)
    return t2
        
############################################################################################################
############################################################################################################

# Testing

# define parameters
target_probability = 0.1
distractor_probabilites = {
    'd1' : 0.1,
    'none': 0.1
} 
sequence_length = 200
prenovelty_standard_stimuli = 1

# create sequence 
new_sequence = Sequence(target_probability,
                        distractor_probabilites,
                        sequence_length,
                        prenovelty_standard_stimuli)
print(new_sequence.getSeq())

print('\n\n')

standard_dev = 1
mean_jitter = 0
max_jitter = 200
min_jitter = -200
scaling_jitter = 200
isi = 1500
psi = 500


new_sequence.expand(isi,psi,standard_dev,max_jitter,min_jitter,mean_jitter,scaling_jitter)

print(new_sequence.getExpandedSeq())




#######################################################################################################
#######################################################################################################


['S', 'd1', 'S', 'S', 'S', 'none', 'S', 'S', 'S', 'S', 'none', 'S', 'none', 'S', 'S', 'd1', 'S', 'T', 'S', 'S', 'none', 'S', 'S', 'S', 'S', 'T', 'S', 'd1', 'S', 'S', 'd1', 'S', 'd1', 'S', 'S', 'T', 'S', 'd1', 'S', 'S', 'S', 'S', 'd1', 'S', 'S', 'S', 'T', 'S', 'S', 'd1', 'S', 'T', 'S', 'S', 'd1', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'none', 'S', 'd1', 'S', 'd1', 'S', 'S', 'T', 'S', 'none', 'S', 'none', 'S', 'S', 'none', 'S', 'T', 'S', 'd1', 'S', 'none', 'S', 'S', 'S', 'none', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'S', 'T', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'none', 'S', 'T', 'S', 'T', 'S', 'S', 'T', 'S', 'S', 'S', 'T', 'S', 'none', 'S', 'd1', 'S', 'S', 'S', 'S', 'T', 'S', 'none', 'S', 'S', 'S', 'S', 'none', 'S', 'none', 'S', 'd1', 'S', 'S', 'S', 'none', 'S', 'none', 'S', 'S', 'S', 'T', 'S', 'S', 'S', 'S', 'S', 'S', 'd1', 'S', 'S', 'S', 'd1', 'S', 'd1', 'S', 'none', 'S', 'S', 'd1', 'S', 'none', 'S', 'S', 'S', 'd1', 'S', 'S', 'S', 'd1', 'S

In [ ]:



class Execute(Sequence):
    
    _music_Dict = {}
    
    def __init__(self,paths_D,*args):
        super().__init__(*args)
        
        self.load_music(paths_D) # ?
        
        
        
    def execute(self): pass
    def load_music(self,path_Dict): pass
    
    
e = Execute({},
            target_probability,
            distractor_probabilites,
            sequence_length,
            prenovelty_standard_stimuli)

print(e.getSeq())

3.41